In [ ]:
# all new rna lines in 21Q3 had wrong internal bam (& sizehash cols) assigned on the tracker
# this notebook fixes 

from __future__ import print_function
import pandas as pd

import os 
from depmapomics import tracker

In [ ]:
# investigate why internal_bam and internal_bai are swapped within a set of samples
track = tracker.getTracker()
track = track.dropna(subset=['internal_bam_filepath', 'internal_bai_filepath'])
track['cds_id'] = track.index

In [ ]:
swappedbams = track[track.apply(lambda x: 'CDS-' in x.internal_bam_filepath and x.cds_id not in x.internal_bam_filepath, axis=1)]

In [ ]:
swappedbams.internal_bam_filepath

In [ ]:
swappedbams.cds_id.tolist()

In [ ]:
# making sure there's no bams missing/broken, just swapped within this set
set(swappedbams.index) == set([x.split('/')[-1][:10] for x in swappedbams['internal_bam_filepath']])

In [ ]:
set(swappedbams.index) == set([x.split('/')[-1][:10] for x in swappedbams['internal_bai_filepath']])

In [ ]:
# pull tracker again, correct internal_bam_filepath, internal_bai_filepath, size, crc32c_hash and md5_hash
track = tracker.getTracker()
for idx in swappedbams.index:
    correct_idx = swappedbams.loc[idx, 'internal_bam_filepath'].split('/')[-1][:10]
    track.loc[correct_idx, 'internal_bam_filepath'] = swappedbams.loc[idx, 'internal_bam_filepath']
    track.loc[correct_idx, 'internal_bai_filepath'] = swappedbams.loc[idx, 'internal_bai_filepath']
    track.loc[correct_idx, 'size'] = swappedbams.loc[idx, 'size']
    track.loc[correct_idx, 'crc32c_hash'] = swappedbams.loc[idx, 'crc32c_hash']
    track.loc[correct_idx, 'md5_hash'] = swappedbams.loc[idx, 'md5_hash']

In [ ]:
sheetname='ccle sample tracker'
sheetcreds='../../.credentials.json'

In [ ]:
from genepy.google.google_sheet import dfToSheet
dfToSheet(track, sheetname, secret=sheetcreds)